# MPC Mag Bands API

#### This tutorial demonstrates how to use the Minor Planet Center's Mag Bands API.

The Mag Bands API returns information about photometric magnitude bands recognized by the MPC. This is useful when you want to:

 - Look up the description of a specific magnitude band code
 - Retrieve the full list of all recognized bands
 - Check the V-band conversion offset for a given band
 - Validate band codes before submitting observations

The Mag Bands API is a REST endpoint. You can send GET requests to:

    https://data.minorplanetcenter.net/api/mag-band

In the examples below we use Python code to query the API.

Further information and documentation can be found at:
 - [Mag Bands API documentation](https://docs.minorplanetcenter.net/mpc-ops-docs/apis/mag-bands/)

# Import Packages
Here we import the standard Python packages needed to call the API and interpret the returned data.

In [ ]:
import requests
import json

# API Parameters

The Mag Bands API accepts GET requests with a JSON body containing a single `band` field:

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `band` | String | Yes | A band code (e.g. `"V"`) or `"all"` to retrieve every defined band |

### Response Fields

| Field | Type | Description |
|-------|------|-------------|
| `band` | String | The band code |
| `notes` | String | Human-readable description of the band |
| `allowed` | Boolean | Whether this band is currently accepted for submissions |
| `v_conversion` | Numeric or null | Offset to convert from this band to V magnitude (null if not defined) |

# API Endpoint

In [ ]:
# URL of the Mag Bands API endpoint
url = "https://data.minorplanetcenter.net/api/mag-band"

# Query a Single Band

Here we query for information about the V (Visual) band. The response includes the original request context and an array of matching results.

In [ ]:
# Query for the "V" (Visual) band
response = requests.get(url, json={"band": "V"})
response.raise_for_status()

print(json.dumps(response.json(), indent=4))

# Query All Bands

To retrieve every magnitude band recognized by the MPC, use `"band": "all"`.

In [ ]:
# Query for all defined bands
response = requests.get(url, json={"band": "all"})
response.raise_for_status()

result = response.json()
bands = result["response"]

print(f"Total bands defined: {len(bands)}\n")
print(f"{'Band':<8} {'Notes':<35} {'Allowed':<10} {'V conv'}")
print("-" * 65)
for entry in bands:
    band = entry['band'] or "(None)"
    notes = entry['notes'] or ""
    v_conv = entry['v_conversion']
    v_str = f"{v_conv:+.2f}" if v_conv is not None else "n/a"
    print(f"{band:<8} {notes:<35} {str(entry['allowed']):<10} {v_str}")

# Query Several Common Bands

The API accepts one band per request. To look up several bands, loop over the codes.

In [ ]:
# Look up a selection of commonly used bands
band_codes = ["V", "R", "B", "g", "r", "i", "w", "G"]

print(f"{'Band':<8} {'Notes':<35} {'V conversion'}")
print("-" * 58)
for code in band_codes:
    resp = requests.get(url, json={"band": code})
    if resp.ok:
        for entry in resp.json().get("response", []):
            band = entry['band'] or "(None)"
            notes = entry['notes'] or ""
            v_conv = entry['v_conversion']
            v_str = f"{v_conv:+.2f}" if v_conv is not None else "n/a"
            print(f"{band:<8} {notes:<35} {v_str}")
    else:
        print(f"{code:<8} (error: {resp.status_code})")

# Unrecognized Band Code

If you query for a band code that is not recognized, the API returns a 200 status with an empty `response` array.

In [ ]:
# Query for a band that does not exist
response = requests.get(url, json={"band": "ZZZ"})

print(f"Status code: {response.status_code}")
print(json.dumps(response.json(), indent=4))

# Helper Function

A convenient wrapper for querying the Mag Bands API.

In [ ]:
def get_mag_band(band):
    """
    Query the MPC Mag Bands API.

    Parameters
    ----------
    band : str
        A single band code (e.g. "V") or "all".

    Returns
    -------
    list of dict
        Each dict has keys: band, notes, allowed, v_conversion.
    """
    response = requests.get(
        "https://data.minorplanetcenter.net/api/mag-band",
        json={"band": band}
    )
    response.raise_for_status()
    return response.json()["response"]


# Example usage
print("Single band lookup:")
print(get_mag_band("V"))
print(f"\nTotal bands defined: {len(get_mag_band('all'))}")

# Summary

The MPC Mag Bands API provides a simple way to look up photometric magnitude bands recognized by the MPC.

Key points:
- **Endpoint**: `https://data.minorplanetcenter.net/api/mag-band`
- **Method**: GET with JSON body
- **Single band**: `{"band": "V"}`
- **All bands**: `{"band": "all"}`
- **Response fields**: `band` (code), `notes` (description), `allowed` (boolean), `v_conversion` (V-mag offset or null)

For more information, see:
 - [Mag Bands API documentation](https://docs.minorplanetcenter.net/mpc-ops-docs/apis/mag-bands/)
 - [MPC Jira Helpdesk](https://mpc-service.atlassian.net/servicedesk/customer/portal/13/create/148) for questions or issues